In [1]:
#RUN THIS CELL 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

In [2]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind
pd.set_option('display.max_columns', 25)

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LassoCV

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import confusion_matrix

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import statsmodels.api as sm
from statsmodels.api import OLS

import seaborn as sns
sns.set()

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time

/Users/williamdrew/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/williamdrew/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/williamdrew/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/williamdrew/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/williamdrew/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/williamd

In [25]:
df = pd.read_csv('Songs/songs0.csv')

# remove "spotify:track:", "spotify:artist:", and "spotify:album:" from uri
df["track_uri"] = df["track_uri"].str[14:]
df["artist_uri"] = df["artist_uri"].str[15:]
df["album_uri"] = df["album_uri"].str[14:]

In [26]:
df.head()

,pid,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
0,0,0,Missy Elliott,0UaMYEvWZi0ZqiDOoHU3YI,2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook
1,0,1,Britney Spears,6I9VzXrHxO9rA9A5euc8Ak,26dSoYclwsYLMAKD3tpOr4,Toxic,0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone
2,0,2,Beyoncé,0WqIKmW4BTrj3eJFmnCKMv,6vWDO969PvNqNYHIOW5v0m,Crazy In Love,25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit)
3,0,3,Justin Timberlake,1AWQoqb9bSvzTjaLralEkT,31TPClRtHm23RisEBtV3X7,Rock Your Body,6QPkyl04rXwTGlGlcYaRoW,267266,Justified
4,0,4,Shaggy,1lzr43nnXAijIGYnCT8M8H,5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot


In [3]:
# https://stackoverflow.com/questions/30557409/spotify-api-post-call-response-415
def initiate_api():    
    client_id = "9cd3dd2ea2cf492ca28ab0247a79d781"
    client_secret = "11c972ad002843e9be5ecc31f022dd6e"
    grant_type = 'client_credentials'
    body_params = {'grant_type' : grant_type}
    url = 'https://accounts.spotify.com/api/token'
    response = requests.post(url, data=body_params, auth = (client_id, client_secret)) 
    client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
    return spotipy.Spotify(client_credentials_manager=client_credentials_manager)
sp = initiate_api()

In [4]:
def songs_in_playlist(df, pid):
    '''
    input: dataframe and playlist ID
    returns: list of all songs on playlist by track_uri 
    '''
    playlist = np.where(df.pid == pid)
    return list(df.iloc[playlist]["track_uri"])

def playlist_means(tracks):
    '''
    input: list of tracks on a playlist (can be generated from songs_in_playlist())
    returns: statistics for audio features of playlist  
    '''
    num = len(tracks)
    # We can only pass 100 tracks at a time
    if num <= 100:
        p = pd.DataFrame.from_records(sp.audio_features(tracks))
    else:
        start = 0
        end = 100
        p = pd.DataFrame.from_records(sp.audio_features(tracks[start:end]))
        for i in range(int(num / 100)):
            start = end
            diff = num - end
            if diff > 100:
                end += 100
            else:
                end += diff
            
            new_p = pd.DataFrame.from_records(sp.audio_features(tracks[start:end]))
            p = p.append(new_p) 
    return p

In [31]:
tracks = songs_in_playlist(df, 20)
playlist_means(tracks)

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0.027000,https://api.spotify.com/v1/audio-analysis/1iDP...,0.914,125217,0.415,1iDPB2WNsGEWuKycQyDGn6,0.000000,1,0.1140,-7.219,1,0.3220,137.974,4,https://api.spotify.com/v1/tracks/1iDPB2WNsGEW...,audio_features,spotify:track:1iDPB2WNsGEWuKycQyDGn6,0.4960
1,0.000404,https://api.spotify.com/v1/audio-analysis/0AIC...,0.659,143969,0.614,0AICBlLzFCTpUqmAbtzB2z,0.000000,9,0.1230,-7.320,0,0.0541,119.405,4,https://api.spotify.com/v1/tracks/0AICBlLzFCTp...,audio_features,spotify:track:0AICBlLzFCTpUqmAbtzB2z,0.2600
2,0.003650,https://api.spotify.com/v1/audio-analysis/1c5L...,0.934,192000,0.407,1c5Lbn9mYwNYbnNiizw6D0,0.000000,0,0.1100,-8.058,1,0.3450,140.024,4,https://api.spotify.com/v1/tracks/1c5Lbn9mYwNY...,audio_features,spotify:track:1c5Lbn9mYwNYbnNiizw6D0,0.0793
3,0.363000,https://api.spotify.com/v1/audio-analysis/4s2B...,0.708,182267,0.612,4s2BWgnSQ9NZiOQM4PP4HB,0.000000,6,0.2040,-8.939,0,0.3020,94.274,4,https://api.spotify.com/v1/tracks/4s2BWgnSQ9NZ...,audio_features,spotify:track:4s2BWgnSQ9NZiOQM4PP4HB,0.3410
4,0.002640,https://api.spotify.com/v1/audio-analysis/7GX5...,0.732,182707,0.750,7GX5flRQZVHRAGd6B4TmDO,0.000000,11,0.1090,-6.366,0,0.2310,155.096,4,https://api.spotify.com/v1/tracks/7GX5flRQZVHR...,audio_features,spotify:track:7GX5flRQZVHRAGd6B4TmDO,0.4010
5,0.057900,https://api.spotify.com/v1/audio-analysis/3uFm...,0.917,122996,0.624,3uFmweGlFudKNiX13XFwFm,0.000000,4,0.0714,-6.728,1,0.1470,124.984,4,https://api.spotify.com/v1/tracks/3uFmweGlFudK...,audio_features,spotify:track:3uFmweGlFudKNiX13XFwFm,0.3430
6,0.015900,https://api.spotify.com/v1/audio-analysis/0Fcz...,0.955,115316,0.773,0FczZscGB1PynHk6zwNO69,0.000172,1,0.0809,-3.062,1,0.2830,135.039,4,https://api.spotify.com/v1/tracks/0FczZscGB1Py...,audio_features,spotify:track:0FczZscGB1PynHk6zwNO69,0.2380
7,0.066700,https://api.spotify.com/v1/audio-analysis/4Cb4...,0.821,191990,0.713,4Cb4Ehfj2S8UhC5JKqmJ6m,0.000000,0,0.0923,-4.814,0,0.2430,160.988,4,https://api.spotify.com/v1/tracks/4Cb4Ehfj2S8U...,audio_features,spotify:track:4Cb4Ehfj2S8UhC5JKqmJ6m,0.6140
8,0.140000,https://api.spotify.com/v1/audio-analysis/7zez...,0.939,145171,0.617,7zez4ZwqfSqD6fPQgcnqwu,0.000000,7,0.1120,-5.933,0,0.2980,131.949,4,https://api.spotify.com/v1/tracks/7zez4ZwqfSqD...,audio_features,spotify:track:7zez4ZwqfSqD6fPQgcnqwu,0.6800
9,0.006130,https://api.spotify.com/v1/audio-analysis/7MCs...,0.691,201721,0.715,7MCs283jdifycrZoMPPynX,0.000000,8,0.0368,-7.215,1,0.0589,77.223,4,https://api.spotify.com/v1/tracks/7MCs283jdify...,audio_features,spotify:track:7MCs283jdifycrZoMPPynX,0.4650


---
**Note**

We could construct a model which determines if a user will like a certain playlist based on comparing their playlists and liked playlists to the audio features of our playlist. Train model on users playlists and likes, classify playlist as 0 or 1 if the user will like the playlist.

I believe our final model will look something like this: https://developer.spotify.com/console/get-recommendations/

In [37]:
for csvnum in np.arange(0, 30):
    df = pd.read_csv('Songs/songs'+str(csvnum)+'.csv')
    songs = pd.DataFrame()
    ranges1 = np.arange(0,1000, 1)
    for n in ranges1:
        try:
            songs = songs.append(playlist_means(songs_in_playlist(df, n)))
        except ValueError:
            print("Playlist n=",str(n + csvnum*1000),"has error")
        except TypeError:
            print("Playlist n=",str(n + csvnum*1000),"has error")
        except IndexError:
            print("Playlist n=",str(n + csvnum*1000),"has error")
        except AttributeError:
            print("Playlist n=",str(n + csvnum*1000),"has error")
        if n%500 == 0:
            print(n + csvnum*1000)
            sp = initiate_api()
    songs.to_pickle("pickles/songs_"+str(csvnum)+".pkl")
    print("Parsed CSV", csvnum)
print("DONE")

0
500
Playlist n= 848 has error
Parsed CSV 0
1000
1500
Playlist n= 1627 has error
Parsed CSV 1
2000
2500
Parsed CSV 2
3000
3500
Parsed CSV 3
4000
4500
Parsed CSV 4
5000
Playlist n= 5157 has error
Playlist n= 5247 has error
Playlist n= 5485 has error
5500
Playlist n= 5859 has error
Playlist n= 5862 has error
Parsed CSV 5
6000
Playlist n= 6022 has error
Playlist n= 6330 has error
6500
Playlist n= 6506 has error
Playlist n= 6925 has error
Parsed CSV 6
7000
Playlist n= 7186 has error
7500
Playlist n= 7560 has error
Parsed CSV 7
8000
Playlist n= 8124 has error
Playlist n= 8477 has error
8500
Parsed CSV 8
9000
9500
Parsed CSV 9
10000
Playlist n= 10238 has error
10500
Parsed CSV 10
11000
Playlist n= 11337 has error
11500
Parsed CSV 11
12000
Playlist n= 12313 has error
12500
Parsed CSV 12
13000
Playlist n= 13038 has error
Playlist n= 13131 has error
13500
Playlist n= 13769 has error
Playlist n= 13912 has error
Parsed CSV 13
14000
14500
Playlist n= 14755 has error
Parsed CSV 14
15000
15500
Pars

## Pickle Aggregator

In [5]:
songs = pd.DataFrame()
for i in range(30):
    songs = songs.append(pd.read_pickle("pickles/songs_"+str(i)+".pkl"))
    
songs.to_pickle("pickles/songs_30k.pkl")

In [13]:
songs = pd.read_pickle("pickles/songs_30k.pkl")

## Delete duplicate songs

In [14]:
songs.drop_duplicates(subset ="id", keep = 'first', inplace = True)

In [15]:
songs = songs.reset_index()

In [16]:
sum(songs['id'] == '6I9VzXrHxO9rA9A5euc8Ak')

1

In [17]:
songs

,index,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0,0.031100,https://api.spotify.com/v1/audio-analysis/0UaM...,0.904,226864,0.8130,0UaMYEvWZi0ZqiDOoHU3YI,0.006970,4,0.0471,-7.105,0,0.1210,125.461,4,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,audio_features,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,0.8100
1,1,0.024900,https://api.spotify.com/v1/audio-analysis/6I9V...,0.774,198800,0.8380,6I9VzXrHxO9rA9A5euc8Ak,0.025000,5,0.2420,-3.914,0,0.1140,143.040,4,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,audio_features,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,0.9240
2,2,0.002380,https://api.spotify.com/v1/audio-analysis/0WqI...,0.664,235933,0.7580,0WqIKmW4BTrj3eJFmnCKMv,0.000000,2,0.0598,-6.583,0,0.2100,99.259,4,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,audio_features,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,0.7010
3,3,0.202000,https://api.spotify.com/v1/audio-analysis/1AWQ...,0.891,267267,0.7140,1AWQoqb9bSvzTjaLralEkT,0.000234,4,0.0521,-6.055,0,0.1400,100.972,4,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,audio_features,spotify:track:1AWQoqb9bSvzTjaLralEkT,0.8180
4,4,0.056100,https://api.spotify.com/v1/audio-analysis/1lzr...,0.853,227600,0.6060,1lzr43nnXAijIGYnCT8M8H,0.000000,0,0.3130,-4.596,1,0.0713,94.759,4,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,audio_features,spotify:track:1lzr43nnXAijIGYnCT8M8H,0.6540
5,5,0.021200,https://api.spotify.com/v1/audio-analysis/0XUf...,0.881,250373,0.7880,0XUfyU2QviPAs6bxSpXYG4,0.000000,2,0.0377,-4.669,1,0.1680,104.997,4,https://api.spotify.com/v1/tracks/0XUfyU2QviPA...,audio_features,spotify:track:0XUfyU2QviPAs6bxSpXYG4,0.5920
6,6,0.257000,https://api.spotify.com/v1/audio-analysis/68vg...,0.662,223440,0.5070,68vgtRHr7iZHpzGpon6Jlo,0.000000,5,0.0465,-8.238,1,0.1180,86.412,4,https://api.spotify.com/v1/tracks/68vgtRHr7iZH...,audio_features,spotify:track:68vgtRHr7iZHpzGpon6Jlo,0.6760
7,7,0.158000,https://api.spotify.com/v1/audio-analysis/3BxW...,0.544,225560,0.8230,3BxWKCI06eQ5Od8TY2JBeA,0.000000,2,0.2680,-4.318,1,0.3200,210.750,4,https://api.spotify.com/v1/tracks/3BxWKCI06eQ5...,audio_features,spotify:track:3BxWKCI06eQ5Od8TY2JBeA,0.4340
8,8,0.273000,https://api.spotify.com/v1/audio-analysis/7H6e...,0.713,271333,0.6780,7H6ev70Weq6DdpZyyTmUXk,0.000000,5,0.1490,-3.525,0,0.1020,138.009,4,https://api.spotify.com/v1/tracks/7H6ev70Weq6D...,audio_features,spotify:track:7H6ev70Weq6DdpZyyTmUXk,0.7340
9,9,0.103000,https://api.spotify.com/v1/audio-analysis/2Ppr...,0.728,235213,0.9740,2PpruBYCo4H7WOBJ7Q2EwM,0.000532,4,0.1750,-2.261,0,0.0665,79.526,4,https://api.spotify.com/v1/tracks/2PpruBYCo4H7...,audio_features,spotify:track:2PpruBYCo4H7WOBJ7Q2EwM,0.9650


In [18]:
songs.to_pickle("pickles/songs_30k_dropped.pkl")